# Introduction to Differential Forwarding Analysis
Intro sketch:
* Network engineers frequently have to make changes to forwarding behavior: add new routes, opening or closing certain flows, moving flows around, etc.
* These changes are hard to get right, and hard to validate.
* This notebook will show how batfish can help validate changes to network forwarding.
* Differential analyses answer questions about and/or validate changes between two snapshots of the network.

In [3]:
# Import packages and load questions
%run startup.py

## Example 1
Intro: describe the network and the change we want to make.

We want to cost out `as2core1` for maintenance.


### Step 1: Test current behavior

In [41]:
EX1_NETWORK_NAME = "differential-example1"
EX1_BASE_NAME = "base"
EX1_BASE_PATH = "networks/differential-ex1-base"

bf_set_network(EX1_NETWORK_NAME)
bf_init_snapshot(EX1_BASE_PATH, name=EX1_BASE_NAME, overwrite=True)

'base'

In [42]:
answer = bfq.reachability(
    headers=HeaderConstraints(dstIps="2.128.0.1", applications=["SSH"]), 
    pathConstraints=PathConstraints(startLocation="enter(as2border.*[GigabitEthernet0/0])")
).answer(snapshot=EX1_BASE_NAME)
display_html(answer.frame())

,Flow,Traces,TraceCount
0,Src IP: 10.12.11.0Src Port: 0Dst IP: 2.128.0.1Dst Port: 22IP Protocol: TCPStart Location: as2border1 interface=GigabitEthernet0/0,"ACCEPTED1. node: as2border1 RECEIVED(GigabitEthernet0/0: OUTSIDE_TO_INSIDE) FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet1/0)2. node: as2core1 RECEIVED(GigabitEthernet0/0) FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet3/0)3. node: as2dist2 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: bgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet2/0)4. node: as2dept1 RECEIVED(GigabitEthernet1/0) ACCEPTED(InboundStep)",1
1,Src IP: 10.23.21.0Src Port: 0Dst IP: 2.128.0.1Dst Port: 22IP Protocol: TCPStart Location: as2border2 interface=GigabitEthernet0/0,"ACCEPTED1. node: as2border2 RECEIVED(GigabitEthernet0/0: OUTSIDE_TO_INSIDE) FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet2/0)2. node: as2core1 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet3/0)3. node: as2dist2 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: bgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet2/0)4. node: as2dept1 RECEIVED(GigabitEthernet1/0) ACCEPTED(InboundStep)",1


### Step 2: Author the change
```
$ diff -r networks/differential-ex1-base networks/differential-ex1-change
diff -r networks/differential-ex1-base/configs/as2core1.cfg networks/differential-ex1-change/configs/as2core1.cfg
71a72
>  ip ospf cost 500
76a78
>  ip ospf cost 500
81a84
>  ip ospf cost 500
111a115
>   neighbor as2 shutdown
```

In [43]:
EX1_CHANGE_NAME = "change"
EX1_CHANGE_PATH = "networks/differential-ex1-change"

bf_init_snapshot(EX1_CHANGE_PATH, name=EX1_CHANGE_NAME, overwrite=True)

'change'

TODO: how does the following relate to the spec?

In [44]:
answer = bfq.reachability(
    headers=HeaderConstraints(dstIps="2.128.0.1", applications=["SSH"]), 
    pathConstraints=PathConstraints(startLocation="enter(as2border.*[GigabitEthernet0/0])")
).answer(snapshot=EX1_CHANGE_NAME)
display_html(answer.frame())

,Flow,Traces,TraceCount


Show that nothing can transit `as2core1` now.

In [54]:
answer = bfq.reachability(
    pathConstraints=PathConstraints(transitLocations="as2core1")
).answer(snapshot=EX1_CHANGE_NAME)
display_html(answer.frame())

answer = bfq.reachability(
    actions = "FAILURE",
    pathConstraints=PathConstraints(transitLocations="as2core1")
).answer(snapshot=EX1_CHANGE_NAME)
display_html(answer.frame())

,Flow,Traces,TraceCount
0,Src IP: 2.1.1.2Src Port: 0Dst IP: 2.1.1.1Dst Port: 0IP Protocol: HOPOPTStart Location: as2border2,"ACCEPTED1. node: as2border2 ORIGINATED(default) FORWARDED(Routes: ospf [Network: 2.1.1.1/32, Next Hop IP:2.12.21.2]) TRANSMITTED(GigabitEthernet2/0)2. node: as2core1 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: ospf [Network: 2.1.1.1/32, Next Hop IP:2.12.11.1]) TRANSMITTED(GigabitEthernet0/0)3. node: as2border1 RECEIVED(GigabitEthernet1/0) ACCEPTED(InboundStep)",1
1,Src IP: 2.1.2.1Src Port: 0Dst IP: 2.1.1.1Dst Port: 0IP Protocol: HOPOPTStart Location: as2core1,"ACCEPTED1. node: as2core1 ORIGINATED(default) FORWARDED(Routes: ospf [Network: 2.1.1.1/32, Next Hop IP:2.12.11.1]) TRANSMITTED(GigabitEthernet0/0)2. node: as2border1 RECEIVED(GigabitEthernet1/0) ACCEPTED(InboundStep)",1
2,Src IP: 2.1.3.1Src Port: 0Dst IP: 2.1.1.1Dst Port: 0IP Protocol: HOPOPTStart Location: as2dist1,"ACCEPTED1. node: as2dist1 ORIGINATED(default) FORWARDED(Routes: ospf [Network: 2.1.1.1/32, Next Hop IP:2.23.11.2]) TRANSMITTED(GigabitEthernet0/0)2. node: as2core1 RECEIVED(GigabitEthernet2/0: blocktelnet) FORWARDED(Routes: ospf [Network: 2.1.1.1/32, Next Hop IP:2.12.11.1]) TRANSMITTED(GigabitEthernet0/0)3. node: as2border1 RECEIVED(GigabitEthernet1/0) ACCEPTED(InboundStep)",1
3,Src IP: 2.1.3.2Src Port: 0Dst IP: 2.1.1.1Dst Port: 0IP Protocol: HOPOPTStart Location: as2dist2,"ACCEPTED1. node: as2dist2 ORIGINATED(default) FORWARDED(Routes: ospf [Network: 2.1.1.1/32, Next Hop IP:2.23.12.2]) TRANSMITTED(GigabitEthernet1/0)2. node: as2core1 RECEIVED(GigabitEthernet3/0: blocktelnet) FORWARDED(Routes: ospf [Network: 2.1.1.1/32, Next Hop IP:2.12.11.1]) TRANSMITTED(GigabitEthernet0/0)3. node: as2border1 RECEIVED(GigabitEthernet1/0) ACCEPTED(InboundStep)",1


,Flow,Traces,TraceCount
0,Src IP: 2.1.1.2Src Port: 0Dst IP: 10.12.11.0Dst Port: 0IP Protocol: HOPOPTStart Location: as2border2,"DENIED_OUT1. node: as2border2 ORIGINATED(default) FORWARDED(Routes: ospfE2 [Network: 10.12.11.0/24, Next Hop IP:2.12.21.2]) TRANSMITTED(GigabitEthernet2/0)2. node: as2core1 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: ospfE2 [Network: 10.12.11.0/24, Next Hop IP:2.12.11.1]) TRANSMITTED(GigabitEthernet0/0)3. node: as2border1 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: connected [Network: 10.12.11.0/24, Next Hop IP:AUTO/NONE(-1l)]) DENIED(GigabitEthernet0/0: INSIDE_TO_AS1)",1
1,Src IP: 2.1.2.1Src Port: 0Dst IP: 10.12.11.0Dst Port: 0IP Protocol: HOPOPTStart Location: as2core1,"DENIED_OUT1. node: as2core1 ORIGINATED(default) FORWARDED(Routes: ospfE2 [Network: 10.12.11.0/24, Next Hop IP:2.12.11.1]) TRANSMITTED(GigabitEthernet0/0)2. node: as2border1 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: connected [Network: 10.12.11.0/24, Next Hop IP:AUTO/NONE(-1l)]) DENIED(GigabitEthernet0/0: INSIDE_TO_AS1)",1
2,Src IP: 2.1.3.1Src Port: 0Dst IP: 10.12.11.0Dst Port: 0IP Protocol: HOPOPTStart Location: as2dist1,"DENIED_OUT1. node: as2dist1 ORIGINATED(default) FORWARDED(Routes: ospfE2 [Network: 10.12.11.0/24, Next Hop IP:2.23.11.2]) TRANSMITTED(GigabitEthernet0/0)2. node: as2core1 RECEIVED(GigabitEthernet2/0: blocktelnet) FORWARDED(Routes: ospfE2 [Network: 10.12.11.0/24, Next Hop IP:2.12.11.1]) TRANSMITTED(GigabitEthernet0/0)3. node: as2border1 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: connected [Network: 10.12.11.0/24, Next Hop IP:AUTO/NONE(-1l)]) DENIED(GigabitEthernet0/0: INSIDE_TO_AS1)",1
3,Src IP: 2.1.3.2Src Port: 0Dst IP: 10.12.11.0Dst Port: 0IP Protocol: HOPOPTStart Location: as2dist2,"DENIED_OUT1. node: as2dist2 ORIGINATED(default) FORWARDED(Routes: ospfE2 [Network: 10.12.11.0/24, Next Hop IP:2.23.12.2]) TRANSMITTED(GigabitEthernet1/0)2. node: as2core1 RECEIVED(GigabitEthernet3/0: blocktelnet) FORWARDED(Routes: ospfE2 [Network: 10.12.11.0/24, Next Hop IP:2.12.11.1]) TRANSMITTED(GigabitEthernet0/0)3. node: as2border1 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: connected [Network: 10.12.11.0/24, Next Hop IP:AUTO/NONE(-1l)]) DENIED(GigabitEthernet0/0: INSIDE_TO_AS1)",1


Sanity check all differences:

In [52]:
pathConstraints = None
headerConstraints = None 
answer = bfq.differentialReachability(pathConstraints=pathConstraints, headers=headerConstraints).answer(
    snapshot=EX1_CHANGE_NAME, 
    reference_snapshot=EX1_BASE_NAME)
display_html(answer.frame())

,Flow,Base_Traces,Base_TraceCount,Delta_Traces,Delta_TraceCount
0,Src IP: 2.1.1.1Src Port: 0Dst IP: 2.23.11.2Dst Port: 23IP Protocol: TCPStart Location: as2border1,"DENIED_IN1. node: as2border1 ORIGINATED(default) FORWARDED(Routes: ospf [Network: 2.23.11.0/24, Next Hop IP:2.12.12.2]) TRANSMITTED(GigabitEthernet2/0)2. node: as2core2 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: ospf [Network: 2.23.11.0/24, Next Hop IP:2.23.21.3]) TRANSMITTED(GigabitEthernet3/0)3. node: as2dist1 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: connected [Network: 2.23.11.0/24, Next Hop IP:AUTO/NONE(-1l)]) TRANSMITTED(GigabitEthernet0/0)4. node: as2core1 DENIED(GigabitEthernet2/0: blocktelnet)",1,"ACCEPTED1. node: as2border1 ORIGINATED(default) FORWARDED(Routes: ospf [Network: 2.23.11.0/24, Next Hop IP:2.12.11.2]) TRANSMITTED(GigabitEthernet1/0)2. node: as2core1 RECEIVED(GigabitEthernet0/0) ACCEPTED(InboundStep)",1
1,Src IP: 2.1.1.2Src Port: 0Dst IP: 2.23.11.2Dst Port: 23IP Protocol: TCPStart Location: as2border2,"DENIED_IN1. node: as2border2 ORIGINATED(default) FORWARDED(Routes: ospf [Network: 2.23.11.0/24, Next Hop IP:2.12.22.2]) TRANSMITTED(GigabitEthernet1/0)2. node: as2core2 RECEIVED(GigabitEthernet0/0) FORWARDED(Routes: ospf [Network: 2.23.11.0/24, Next Hop IP:2.23.21.3]) TRANSMITTED(GigabitEthernet3/0)3. node: as2dist1 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: connected [Network: 2.23.11.0/24, Next Hop IP:AUTO/NONE(-1l)]) TRANSMITTED(GigabitEthernet0/0)4. node: as2core1 DENIED(GigabitEthernet2/0: blocktelnet)",1,"ACCEPTED1. node: as2border2 ORIGINATED(default) FORWARDED(Routes: ospf [Network: 2.23.11.0/24, Next Hop IP:2.12.21.2]) TRANSMITTED(GigabitEthernet2/0)2. node: as2core1 RECEIVED(GigabitEthernet1/0) ACCEPTED(InboundStep)",1
2,Src IP: 2.1.2.1Src Port: 0Dst IP: 2.128.0.0Dst Port: 0IP Protocol: HOPOPTStart Location: as2core1,NO_ROUTE1. node: as2core1 ORIGINATED(default) NO_ROUTE(Routes: ),1,"EXITS_NETWORK1. node: as2core1 ORIGINATED(default) FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet3/0)2. node: as2dist2 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: bgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet2/0)3. node: as2dept1 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: connected [Network: 2.128.0.0/24, Next Hop IP:AUTO/NONE(-1l)]) EXITS_NETWORK(GigabitEthernet2/0)",1
3,Src IP: 2.1.3.1Src Port: 0Dst IP: 2.1.1.2Dst Port: 23IP Protocol: TCPStart Location: as2dist1,"ACCEPTED1. node: as2dist1 ORIGINATED(default) FORWARDED(Routes: ospf [Network: 2.1.1.2/32, Next Hop IP:2.23.21.2]) TRANSMITTED(GigabitEthernet1/0)2. node: as2core2 RECEIVED(GigabitEthernet3/0) FORWARDED(Routes: ospf [Network: 2.1.1.2/32, Next Hop IP:2.12.22.1]) TRANSMITTED(GigabitEthernet0/0)3. node: as2border2 RECEIVED(GigabitEthernet1/0) ACCEPTED(InboundStep)",1,"DENIED_IN1. node: as2dist1 ORIGINATED(default) FORWARDED(Routes: ospf [Network: 2.1.1.2/32, Next Hop IP:2.23.11.2]) TRANSMITTED(GigabitEthernet0/0)2. node: as2core1 DENIED(GigabitEthernet2/0: blocktelnet)",1
4,Src IP: 2.1.3.2Src Port: 0Dst IP: 2.1.1.2Dst Port: 23IP Protocol: TCPStart Location: as2dist2,"ACCEPTED1. node: as2dist2 ORIGINATED(default) FORWARDED(Routes: ospf [Network: 2.1.1.2/32, Next Hop IP:2.23.22.2]) TRANSMITTED(GigabitEthernet0/0)2. node: as2core2 RECEIVED(GigabitEthernet2/0) FORWARDED(Routes: ospf [Network: 2.1.1.2/32, Next Hop IP:2.12.22.1]) TRANSMITTED(GigabitEthernet0/0)3. node: as2border2 RECEIVED(GigabitEthernet1/0) ACCEPTED(InboundStep)",1,"DENIED_IN1. node: as2dist2 ORIGINATED(default) FORWARDED(Routes: ospf [Network: 2.1.1.2/32, Next Hop IP:2.23.12.2]) TRANSMITTED(GigabitEthernet1/0)2. node: as2core1 DENIED(GigabitEthernet3/0: blocktelnet)",1


## Example 2

Describe the network and the desired change: deny SSH from outside the AS.


## Step 1: Test current behavior

In [30]:
EX2_NETWORK_NAME = "differential-example2"
EX2_BASE_NAME = "base"
EX2_BASE_PATH = "networks/differential-ex2-base"

bf_set_network(EX2_NETWORK_NAME)
bf_init_snapshot(EX2_BASE_PATH, name=EX2_BASE_NAME, overwrite=True)

'base'

In [32]:
answer = bfq.reachability(
    pathConstraints = PathConstraints(startLocation="enter(as2border.*[GigabitEthernet0/0])", endLocation="host.*"),
    headers = HeaderConstraints(applications="SSH")
).answer(snapshot=EX2_BASE_NAME)
display_html(answer.frame())

Flow  \
0  start=as2border1 interface=GigabitEthernet0/0 [10.12.11.0:0->2.128.0.101:22 TCP]   
1  start=as2border2 interface=GigabitEthernet0/0 [10.23.21.0:0->2.128.0.101:22 TCP]   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     Traces  \
0  [((RECEIVED(GigabitEthernet0/0: OUTSIDE_TO_INSIDE), FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.101.4],ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]), TRANSMITTED(GigabitEthernet1/0)), (RECEIVED(GigabitEthernet0/0), FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.101.4]), TRANSMITTED(GigabitEthernet2/0)), (RECEIVED(GigabitEthernet0/0), FORWARDED(Routes: bgp [Network: 2.128.0.0/24, Next Hop IP:2.34.101.4]), TRANSMITTED(GigabitEthernet2/0)), (RECEIVED(GigabitEthernet0/0), FORWARDED(Routes: connected [Network: 2.128.0.0/24, Next Hop IP:AUTO/NONE(-1l)]), TRANSMITTED(GigabitEthernet2/0)), (RECEIVED(eth0: filter::INPUT), ACCEPTED(InboundStep))), ((RECEIVED(GigabitEthernet0/0: OUTSIDE_TO_INSIDE), FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.101.4],ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]), TRANSMITTED(GigabitEthernet1/0)), (RECEIVED(GigabitEthernet0/0), FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]), TRANSMITTED(GigabitEthernet3/0)), (RECEIV

These results show that SSH traffic can indeed currently reach the hosts from outside the AS.

## Step 2: Author the change


```
$ diff -r networks/differential-ex2-base networks/differential-ex2-change
diff -r networks/differential-ex2-base/configs/as2border1.cfg networks/differential-ex2-change/configs/as2border1.cfg
123a124
>  deny   tcp any 2.0.0.0 0.255.255.255 eq 22
```

In [26]:
EX2_CHANGE_NAME = "change"
EX2_CHANGE_PATH = "networks/differential-ex2-change"

bf_init_snapshot(EX2_CHANGE_PATH, name=EX2_CHANGE_NAME, overwrite=True)

'change'

In [29]:
answer = bfq.reachability(
    pathConstraints = PathConstraints(
        startLocation="enter(as2border.*[GigabitEthernet0/0])", 
        endLocation="host.*"),
    headers = HeaderConstraints(applications="SSH")
).answer(snapshot=EX2_CHANGE_NAME)
display_html(answer.frame())

Flow  \
0  start=as2border2 interface=GigabitEthernet0/0 [10.23.21.0:0->2.128.0.101:22 TCP]   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     Traces  \
0  [((RECEIVED(GigabitEthernet0/0: OUTSIDE_TO_INSIDE), FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.101.4],ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]), TRANSMITTED(GigabitEthernet1/0)), (RECEIVED(GigabitEthernet0/0), FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]), TRANSMITTED(GigabitEthernet2/0)), (RECEIVED(GigabitEthernet0/0), FORWARDED(Routes: bgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]), TRANSMITTED(GigabitEthernet2/0)), (RECEIVED(GigabitEthernet1/0), FORWARDED(Routes: connected [Network: 2.128.0.0/24, Next Hop IP:AUTO/NONE(-1l)]), TRANSMITTED(GigabitEthernet2/0)), (RECEIVED(eth0: filter::INPUT), ACCEPTED(InboundStep))), ((RECEIVED(GigabitEthernet0/0: OUTSIDE_TO_INSIDE), FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.101.4],ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]), TRANSMITTED(GigabitEthernet1/0)), (RECEIVED(GigabitEthernet0/0), FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.101.4]), TRANSMITTED(GigabitEthernet3/0)), (RECEIVED(GigabitEthernet1/0), FORWARDED(Routes: bgp [Network: 2.128.0.0/24, Next Hop IP:2.34.